## Importing the packages

Importing those packages, that were used in building, training and testing the neural network.

...and maybe some more, too.

In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, Dataset
from torchvision import datasets
from torchvision import transforms
import os
from os import listdir
from os.path import isfile, join
import cv2
from PIL import Image
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as fn
from google.colab import drive

## Importing and loading the data

Somehow the zip files, that were uploaded to Google Drive, were always corrupted, thus we could not mount a drive folder to share the images, we had to upload the files locally from the PC


Unzipping the files

*   the dataset was uploaded into the content folder, then a new folder was created for it by the name 'dataset'

In [6]:
!unzip '/content/scipy.zip' -d '/content/dataset'

Archive:  /content/scipy.zip
  inflating: /content/dataset/test/0_test_ (47).jpg  
  inflating: /content/dataset/test/0_test_ (16).jpg  
  inflating: /content/dataset/test/0_test_ (45).jpg  
  inflating: /content/dataset/test/0_test_ (48).jpg  
  inflating: /content/dataset/test/0_test_ (84).jpg  
  inflating: /content/dataset/test/0_test_ (41).jpg  
  inflating: /content/dataset/test/0_test_ (18).jpg  
  inflating: /content/dataset/test/0_test_ (40).jpg  
  inflating: /content/dataset/test/0_test_ (19).jpg  
  inflating: /content/dataset/test/0_test_ (81).jpg  
  inflating: /content/dataset/test/0_test_ (38).jpg  
  inflating: /content/dataset/test/0_test_ (8).jpg  
  inflating: /content/dataset/test/0_test_ (10).jpg  
  inflating: /content/dataset/test/0_test_ (80).jpg  
  inflating: /content/dataset/test/0_test_ (21).jpg  
  inflating: /content/dataset/test/0_test_ (39).jpg  
  inflating: /content/dataset/test/0_test_ (1).jpg  
  inflating: /content/dataset/test/0_test_ (44).jpg  
 

Preprocess:


*   transforming all the images to Tensor
*   resizing all the images to the same size: 128*128
*   normalising them with mean and standard deviation (the values are looked up from the internet)

This function is called in the next section, while we iterated through the test data



In [7]:
from torchvision.transforms.transforms import ToTensor
preprocess = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((128, 128)),
     transforms.Normalize((0.6092, 0.5814, 0.5933), (0.0765, 0.0499, 0.0608))]
)

Loading the training set
*   after loading the training data set
*   we have itearted through the folders and categorised the elements, whether they are non-neurons or neurons
*   the previous preprocess function is called here and applied on all the images

There are some checkpoints, where we checked the processed elements and their features
*   561 images in the training set
*   all of them have 3 channels, which means they are RGB
*   all of them are preprocessed, thus their size is 128*128

In [8]:
train_datasource = os.listdir(path = '/content/dataset')
train_datasource.sort()
print(train_datasource)
train_datasource = train_datasource[:2] #here we need only the first two folders, which conatin non-neurons [0] and neuorns [0]. 
                                        #the 3rd folder is the test set, which will be used later
print(train_datasource)
idx_neuron = 0 

train_img = []
train_cat = [] #the categories are stored here; 0 or 1

for dir_name in train_datasource:
    images = os.listdir('/content/dataset/' + dir_name) #loading the data into the train_img and train_cat from the first 2 folders
    print(f'current file: {images}')
    images.sort()
    for img_name in images:
      if img_name.find('.jpg') != -1:
        if dir_name == '0':
          train_cat.append(torch.tensor(0))
        else:
          train_cat.append(torch.tensor(1))
        img = cv2.imread(f'/content/dataset/{dir_name}/{img_name}')
        img = np.asarray(img)
        img = preprocess(img)
        train_img.append(img)
      idx_neuron += 1

train_img = torch.stack(train_img, dim=0)
train_cat = torch.stack(train_cat, dim = 0)
train_img.shape, train_cat.shape 

['0', '1', 'test']
['0', '1']
current file: ['0_training_ (315).jpg', '0_training_ (115).jpg', '0_training_ (272).jpg', '0_training_ (345).jpg', '0_training_ (265).jpg', '0_training_ (281).jpg', '0_training_ (122).jpg', '0_training_ (256).jpg', '0_training_ (96).jpg', '0_training_ (266).jpg', '0_training_ (154).jpg', '0_training_ (208).jpg', '0_training_ (353).jpg', '0_training_ (224).jpg', '0_training_ (344).jpg', '0_training_ (57).jpg', '0_training_ (8).jpg', '0_training_ (28).jpg', '0_training_ (313).jpg', '0_training_ (52).jpg', '0_training_ (21).jpg', '0_training_ (27).jpg', '0_training_ (127).jpg', '0_training_ (286).jpg', '0_training_ (295).jpg', '0_training_ (114).jpg', '0_training_ (306).jpg', '0_training_ (155).jpg', '0_training_ (14).jpg', '0_training_ (143).jpg', '0_training_ (329).jpg', '0_training_ (274).jpg', '0_training_ (277).jpg', '0_training_ (7).jpg', '0_training_ (203).jpg', '0_training_ (20).jpg', '0_training_ (218).jpg', '0_training_ (245).jpg', '0_training_ (41)

(torch.Size([561, 3, 128, 128]), torch.Size([561]))

Defining the dataset

In [9]:
class MyDataset(Dataset):
  def __init__(self, train_img, train_cat):
    self.data = train_img
    self.label = train_cat

  def __getitem__(self, index):
    x = self.data[index]
    y = self.label[index]
    return x, y

  def __len__(self):
    return len(self.data)


In [10]:
train_dataset = MyDataset(train_img, train_cat) #creating the dataset by concatenating the images to their labels
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True) #the trainoader has the images shuffled, so the network won't learn the order of the images and labels

## Neural Network

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Training
*   we used a VGG16 neural network with 5 layers

In [12]:
Net = nn.Sequential(         
  nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1), #128
  nn.SELU(),
  #vggblock1
  nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1), #128
  nn.SELU(),
  nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1), #128
  nn.SELU(),
  #vggblock2
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1), 
  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock3
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock4
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  #vggblock5
  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
  nn.SELU(),

  # [(W−K+2P)/S]+1 W: input, K: kernel, P: padding, S: stride
  #helper to check if the calculations for the layers were correct

  nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
  ##classifier part
  nn.Flatten(),
  nn.Linear(in_features=8192, out_features=4096),
  nn.ReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=4096, out_features=2048),
  nn.ReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=2048, out_features=1024),
  nn.ReLU(),
  nn.Dropout(p=0.2),
  nn.Linear(in_features=1024, out_features=2)
)

In [13]:
Net = Net.to(device)

Optimizer + Loss
*   for the Criterion we used Cross Entropy Loss
*   the optimiser is ADAM with learning rate e^(-4)
*   20 epochs were set for this amount of images

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(Net.parameters(), lr=1e-4)
num_epochs = 20
Net.train()
losses = []

for epoch in range(num_epochs):
  idx = 0
  for data, label in train_loader:
    idx += 1
    data = data.cuda()
    label = label.cuda()
    prediction = Net(data)
    loss = criterion(prediction, label)
    losses.append(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # if idx % 20 ==0:
    #   print(f"At iteration {idx} the current loss is: {float(loss)}")
  print(f"At epoch {epoch} the current loss is: {float(loss)}")

At epoch 0 the current loss is: 0.0
At epoch 1 the current loss is: 0.0
At epoch 2 the current loss is: 1.8138039112091064
At epoch 3 the current loss is: 0.0008189899963326752
At epoch 4 the current loss is: 0.0
At epoch 5 the current loss is: 0.0
At epoch 6 the current loss is: 1.1259446144104004
At epoch 7 the current loss is: 5.8412379075889476e-06
At epoch 8 the current loss is: 0.6856203675270081
At epoch 9 the current loss is: 0.683012843132019
At epoch 10 the current loss is: 0.2954636812210083
At epoch 11 the current loss is: 0.5910869836807251
At epoch 12 the current loss is: 0.0009696549386717379
At epoch 13 the current loss is: 3.576278118089249e-07
At epoch 14 the current loss is: 0.0
At epoch 15 the current loss is: 0.0001113352773245424
At epoch 16 the current loss is: 0.0
At epoch 17 the current loss is: 0.0
At epoch 18 the current loss is: 4.7444173105759546e-05
At epoch 19 the current loss is: 9.536738616588991e-07


In [30]:
Net.eval()
data, label = train_dataset[0]
data.unsqueeze(dim=0).shape, label

(torch.Size([1, 3, 128, 128]), tensor(0))

In [31]:
pred = Net(data.unsqueeze(dim=0).cuda())
pred.argmax(-1) == label

tensor([True], device='cuda:0')

Testing

*   similar method and process as it was for the training
*   we checked the attrubures; 139 images, all of them are RGB and their size is 128*128 due to the preprocess function



In [32]:
test_datasource = os.listdir(path = '/content/dataset/test')
test_datasource.sort()
#test_datasource = test_datasource[]
print(test_datasource)
types = [str, int]

test_img = []
test_cat = []
file_names = []

for img_name in test_datasource: #loading the test images similarly to laoding the training images
  if img_name.find('.jpg') != -1:
    if img_name[0] == '0':
      test_cat.append(torch.tensor(0))
    else:
      test_cat.append(torch.tensor(1))
    img = cv2.imread(f'/content/dataset/test/{img_name}')
    img = np.asarray(img)
    #print(img.shape, img_name)
    img = preprocess(img) 
    #print(img.size)
    file_names.append(img_name)
    test_img.append(img)

test_img = torch.stack(test_img, dim=0)
test_cat = torch.stack(test_cat, dim = 0)
test_img.shape, test_cat.shape 

['0_test_ (1).jpg', '0_test_ (10).jpg', '0_test_ (12).jpg', '0_test_ (13).jpg', '0_test_ (14).jpg', '0_test_ (15).jpg', '0_test_ (16).jpg', '0_test_ (17).jpg', '0_test_ (18).jpg', '0_test_ (19).jpg', '0_test_ (2).jpg', '0_test_ (20).jpg', '0_test_ (21).jpg', '0_test_ (22).jpg', '0_test_ (23).jpg', '0_test_ (24).jpg', '0_test_ (25).jpg', '0_test_ (26).jpg', '0_test_ (27).jpg', '0_test_ (28).jpg', '0_test_ (29).jpg', '0_test_ (3).jpg', '0_test_ (30).jpg', '0_test_ (31).jpg', '0_test_ (32).jpg', '0_test_ (33).jpg', '0_test_ (34).jpg', '0_test_ (35).jpg', '0_test_ (36).jpg', '0_test_ (37).jpg', '0_test_ (38).jpg', '0_test_ (39).jpg', '0_test_ (4).jpg', '0_test_ (40).jpg', '0_test_ (41).jpg', '0_test_ (42).jpg', '0_test_ (43).jpg', '0_test_ (44).jpg', '0_test_ (45).jpg', '0_test_ (46).jpg', '0_test_ (47).jpg', '0_test_ (48).jpg', '0_test_ (49).jpg', '0_test_ (5).jpg', '0_test_ (50).jpg', '0_test_ (51).jpg', '0_test_ (52).jpg', '0_test_ (53).jpg', '0_test_ (54).jpg', '0_test_ (55).jpg', '0_t

(torch.Size([139, 3, 128, 128]), torch.Size([139]))

In [33]:
test_dataset = MyDataset(test_img, test_cat)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

## Results

Evaluating the Results
*   here only the calculations occur, the results are shown in the next block
*   we printed the number of observed elements; it must be equal to the previous check point

In [34]:
counter, correct = 0, 0
Net.eval()
for data, label in test_loader: #evaluating the results of the prediction: comparing the predicted to the true labels
  prediction = Net(data.cuda())
  if prediction.argmax() == label.cuda():
    correct += 1
  counter += 1

print(counter)
#print(counter/train_x.shape[0])
accuracy = (correct/counter)*100


139


Displaying the Results on Colab Panels
*   to check, whether the network was correct we did the following: the image name starts with its category, and the predicition and the ground truth appears next to it
*   the last line shows the accuracy of the network, which was calculated in the previous block

In [36]:
i = 0
for data, label in test_loader:
  data = data.cuda()
  label = label.cuda()

  output = Net(data.cuda())
  prediction = torch.argmax(output[0])

  print(f'image: {file_names[i]}, prediction: {str(prediction.item())}, truth: {file_names[i][0]}')
  i += 1

print(f'accuracy: {accuracy:.2f}%')

image: 0_test_ (1).jpg, prediction: 0, truth: 0
image: 0_test_ (10).jpg, prediction: 0, truth: 0
image: 0_test_ (12).jpg, prediction: 0, truth: 0
image: 0_test_ (13).jpg, prediction: 0, truth: 0
image: 0_test_ (14).jpg, prediction: 0, truth: 0
image: 0_test_ (15).jpg, prediction: 0, truth: 0
image: 0_test_ (16).jpg, prediction: 0, truth: 0
image: 0_test_ (17).jpg, prediction: 0, truth: 0
image: 0_test_ (18).jpg, prediction: 0, truth: 0
image: 0_test_ (19).jpg, prediction: 0, truth: 0
image: 0_test_ (2).jpg, prediction: 0, truth: 0
image: 0_test_ (20).jpg, prediction: 0, truth: 0
image: 0_test_ (21).jpg, prediction: 0, truth: 0
image: 0_test_ (22).jpg, prediction: 0, truth: 0
image: 0_test_ (23).jpg, prediction: 0, truth: 0
image: 0_test_ (24).jpg, prediction: 0, truth: 0
image: 0_test_ (25).jpg, prediction: 0, truth: 0
image: 0_test_ (26).jpg, prediction: 0, truth: 0
image: 0_test_ (27).jpg, prediction: 0, truth: 0
image: 0_test_ (28).jpg, prediction: 0, truth: 0
image: 0_test_ (29).jp